# Download BERPublicsearch.zip, unzip & convert to parquet

In [ ]:
# If running this in Google Colab
# If prompted click the `RESTART RUNTIME` button below this cell
# !pip install berpublicsearch numexpr

In [ ]:
# Register your email address with SEAI at https://ndber.seai.ie/BERResearchTool/Register/Register.aspx
email_address="rowan.molony@codema.ie"

In [ ]:
from berpublicsearch.download import download_berpublicsearch_parquet

download_berpublicsearch_parquet(email_address)

# Experiment with Filters

Filters developed by [UCC MaREI Group](https://www.marei.ie/) and [TUDublin](https://www.tudublin.ie/) researchers [Tomas Mac Uidir](https://www.researchgate.net/profile/Tomas_Mac_Uidhir), [Fionn Rogan](https://www.researchgate.net/profile/Fionn_Rogan), [Jason Mc Guire](https://www.researchgate.net/profile/Jason_Mc_Guire) and [Ciara Ahern](https://www.researchgate.net/profile/Ciara_Ahern)

In [ ]:
import dask.dataframe as dd

ber = dd.read_parquet("BERPublicsearch_parquet")

# Create a `TotalFloorArea` column

<span style="color:red"> ... there's more 0 FloorArea buildings than there are 0 GroundFloorArea buildings SO FloorArea is not a reliable column and can be derived instead from:</span> 

```
TotalFloorArea = GroundFloorArea + FirstFloorArea + SecondFloorArea + ThirdFloorArea
```

Try:

```ber.query("`FloorArea` <= 0").compute()```

vs 

```ber.query("`GroundFloorArea` <= 0").compute()```

In [ ]:
ber["TotalFloorArea"] = ber.eval("GroundFloorArea + FirstFloorArea + SecondFloorArea + ThirdFloorArea")

# Check impact of each individual filter

TypeofRating IS ‘Provisional’

In [ ]:
ber.query("`TypeofRating` == 'Provisional    '").compute()

TotalFloorArea <= 0 m^2

In [ ]:
ber.query("`TotalFloorArea` <= 0").compute()

GroundFloorArea < 30m^2 or  GroundFloorArea > 1000m2 

In [ ]:
ber.query("`GroundFloorArea` < 30 or `GroundFloorArea` > 1000").compute()

TotalFloorArea > 500 m2 AND DwellingTypeDescr IN ('End of terrace house','Mid-terrace house','Top-floor apartment','Mid-floor apartment','Basement Dwelling','Apartment','Semi-detached house','Maisonette','Ground-floor apartment') 

In [ ]:
ber.query("`TotalFloorArea` > 500 and `DwellingTypeDescr` == ['End of terrace house','Mid-terrace house','Top-floor apartment','Mid-floor apartment','Basement Dwelling','Apartment','Semi-detached house','Maisonette','Ground-floor apartment']").compute()

LivingAreaPercent > 90 or LivingAreaPercent < 5 

In [ ]:
ber.query("`LivingAreaPercent` > 90 or `LivingAreaPercent` < 5").compute()

HSMainSystemEfficiency ≤ 19

In [ ]:
ber.query("`HSMainSystemEfficiency` <= 19").compute()

HSEffAdjFactor < 0.7 

In [ ]:
ber.query("`HSEffAdjFactor` < 0.7").compute()

WHMainSystemEff > 320 or WHMainSystemEff < 19 

In [ ]:
ber.query("`WHMainSystemEff` > 320 or `WHMainSystemEff` < 19").compute()

WHEffAdjFactor < 0.7 

In [ ]:
ber.query("`WHEffAdjFactor` < 0.7").compute()

HSSupplSystemEff < 19 

In [ ]:
ber.query("`HSSupplSystemEff` < 19").compute()

HSSupplHeatFraction ⊄ (0,0.1,0.15,0.2)

<span style="color:red"> ... both [query and masking](https://pandas.pydata.org/docs/user_guide/indexing.html) are only grabbing out 0 values and not [0.1,0.15,0.2] </span> 


In [ ]:
ber_heatfrac = ber.query("HSSupplHeatFraction not in [0, 0.10, 0.15, 0.20]").compute()

In [ ]:
ber_heatfrac = ber[~ber["HSSupplHeatFraction"].isin([0, 0.10, 0.15, 0.20])].compute()

In [ ]:
ber["HSSupplHeatFraction"].value_counts().compute()

In [ ]:
ber_heatfrac["HSSupplHeatFraction"].value_counts()

DeclaredLossFactor > 20 

In [ ]:
ber.query("`DeclaredLossFactor` > 20").compute()

ThermalBridgingFactor < 0 or ThermalBridgingFactor > 0.15 

In [ ]:
ber.query("`ThermalBridgingFactor` < 0 or `ThermalBridgingFactor` > 0.15").compute()

# Check combined impact of filters

[x] `TypeofRating` != 'Provisional    ' \
[x] `TotalFloorArea` > 0 \
[x] `GroundFloorArea` > 30 and `GroundFloorArea` < 1000 \
[x] `TotalFloorArea` < 500 and `DwellingTypeDescr` == ['End of terrace house','Mid-terrace house','Top-floor apartment','Mid-floor apartment','Basement Dwelling','Apartment','Semi-detached house','Maisonette','Ground-floor apartment'] \
[x] `LivingAreaPercent` < 90 or `LivingAreaPercent` > 5 \
[x] `HSMainSystemEfficiency` > 19 \
[x] `HSEffAdjFactor` > 0.7 \
[x] `WHMainSystemEff` < 320 or `WHMainSystemEff` > 19 \
[x] `WHEffAdjFactor` > 0.7 \
[x] `HSSupplSystemEff` > 19 \
[ ] `HSSupplHeatFraction` in [0.00,0.10,0.15,0.20] \
[x] `DeclaredLossFactor` < 20 \
[x] `ThermalBridgingFactor` > 0 or `ThermalBridgingFactor` <= 0.15 \

In [ ]:
ber_filtered = (
    ber.query("`TypeofRating` != 'Provisional    '")
    .query("`TotalFloorArea` > 0")
    .query("`GroundFloorArea` > 30 and `GroundFloorArea` < 1000")
    .query("`TotalFloorArea` < 500 and `DwellingTypeDescr` == ['End of terrace house','Mid-terrace house','Top-floor apartment','Mid-floor apartment','Basement Dwelling','Apartment','Semi-detached house','Maisonette','Ground-floor apartment']")
    .query("`LivingAreaPercent` < 90 or `LivingAreaPercent` > 5")
    .query("`HSMainSystemEfficiency` > 19")
    .query("`HSEffAdjFactor` > 0.7")
    .query("`WHMainSystemEff` < 320 or `WHMainSystemEff` > 19")
    .query("`WHEffAdjFactor` > 0.7")
    .query("`HSSupplSystemEff` > 19")
#     .loc[ber["HSSupplHeatFraction"].astype(float).isin([0, 0.10, 0.15, 0.20])]
    .query("`DeclaredLossFactor` < 20")
    .query("`ThermalBridgingFactor` > 0 or `ThermalBridgingFactor` <= 0.15")
    .compute()
)

In [ ]:
len(ber) 

In [ ]:
len(ber_filtered)

In [ ]:
percentage_reduction = 100 * (len(ber) - len(ber_filtered)) / len(ber)

In [ ]:
display(f"Filtering eliminated {percentage_reduction}%")